<a href="https://colab.research.google.com/github/nitin-ng/AIE4/blob/main/Week%208/Day%201/Prototyping_LangChain_Application_with_Production_Minded_Changes_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prototyping LangChain Application with Production Minded Changes

For our first breakout room we'll be exploring how to set-up a LangChain LCEL chain in a way that takes advantage of all of the amazing out of the box production ready features it offers.

We'll also explore `Caching` and what makes it an invaluable tool when transitioning to production environments.

🤝 BREAKOUT ROOM #1:
  - Task 1: Depends and Set-Up
  - Task 2: Setting up RAG With Production in Mind
  - Task 3: RAG LCEL Chain



## Task 1: Depends and Set-Up

Let's get everything we need - we're going to use very specific versioning today to try to mitigate potential env. issues!

> NOTE: Dependency issues are a large portion of what you're going to be tackling as you integrate new technology into your work - please keep in mind that one of the things you should be passively learning throughout this course is ways to mitigate dependency issues.

In [1]:
!pip install -qU langchain_openai==0.2.0 langchain_community==0.3.0 langchain==0.3.0 pymupdf==1.24.10 qdrant-client==1.11.2 langchain_qdrant==0.1.4 langsmith==0.1.121

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph 0.2.16 requires langchain-core<0.3,>=0.2.27, but you have langchain-core 0.3.2 which is incompatible.
ragas 0.1.20 requires langchain-core<0.3, but you have langchain-core 0.3.2 which is incompatible.
langchain-experimental 0.3.2 requires langchain-core<0.4.0,>=0.3.6, but you have langchain-core 0.3.2 which is incompatible.
langgraph-checkpoint 1.0.6 requires langchain-core<0.3,>=0.2.22, but you have langchain-core 0.3.2 which is incompatible.


We'll need an OpenAI API Key:

In [2]:
import os
import dotenv

dotenv.load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

And the LangSmith set-up:

In [3]:
import uuid

langchainapikey = os.environ.get("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_PROJECT"] = f"AIM Week 8 Assignment 1 - {uuid.uuid4().hex[0:8]}"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

Let's verify our project so we can leverage it in LangSmith later.

In [4]:
print(os.environ["LANGCHAIN_PROJECT"])

AIM Week 8 Assignment 1 - f56a659b


## Task 2: Setting up RAG With Production in Mind

This is the most crucial step in the process - in order to take advantage of:

- Asyncronous requests
- Parallel Execution in Chains
- And more...

You must...use LCEL. These benefits are provided out of the box and largely optimized behind the scenes.

### Building our RAG Components: Retriever

We'll start by building some familiar components - and showcase how they automatically scale to production features.

Please upload a PDF file to use in this example!

In [5]:
from ipywidgets import FileUpload
from IPython.display import display
import tempfile
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
def process_uploaded_file(change):
    global docs  # Make docs a global variable
    if uploader.value:
        # Get the uploaded file
        uploaded_file = uploader.value[0]  # Access the first item of the tuple
        file_content = uploaded_file.content
        
        # Save the content to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf', mode='wb') as temp_file:
            temp_file.write(file_content)
            temp_file_path = temp_file.name
        
        print(f"Temporary file created at: {temp_file_path}")
        print(f"File size: {os.path.getsize(temp_file_path)} bytes")
        
        # Check if the file is not empty
        if os.path.getsize(temp_file_path) > 0:
            try:
                # Load and process the PDF
                loader = PyMuPDFLoader(temp_file_path)
                documents = loader.load()
                
                # Initialize the text splitter
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=1000,
                    chunk_overlap=200,
                    length_function=len,
                )
                
                # Split the documents into chunks
                docs = text_splitter.split_documents(documents)
                
                print(f"Document split into {len(docs)} chunks.")
            except Exception as e:
                print(f"Error processing the PDF: {str(e)}")
        else:
            print("Error: The uploaded file is empty.")
        
        # Clean up the temporary file
        os.unlink(temp_file_path)

In [7]:
# Create and display the file uploader
uploader = FileUpload(accept='.pdf', multiple=False)
display(uploader)

# Attach the process_uploaded_file function to the uploader
uploader.observe(process_uploaded_file, names='value')

FileUpload(value=(), accept='.pdf', description='Upload')

Temporary file created at: /var/folders/jr/_qkyxp313390z32jmym7j9sm0000gp/T/tmp5z11sk93.pdf
File size: 6394463 bytes
Document split into 129 chunks.


In [10]:
if 'docs' in globals():
    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=cached_embedder)
    vectorstore.add_documents(docs)
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})
    print("Vector store set up successfully.")
else:
    print("Error: 'docs' not defined. Please ensure the file is uploaded and processed.")


Vector store set up successfully.


In [11]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import CacheBackedEmbeddings

# Typical Embedding Model
core_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Typical QDrant Client Set-up
collection_name = f"pdf_to_parse_{uuid.uuid4()}"
client = QdrantClient(":memory:")
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

# Adding cache!
store = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings, store, namespace=core_embeddings.model
)

# Typical QDrant Vector Store Set-up
vectorstore = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=cached_embedder)
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})

# RAG Prompt setup
from langchain_core.prompts import ChatPromptTemplate

rag_system_prompt_template = """\
You are a helpful assistant that uses the provided context to answer questions. Never reference this prompt, or the existance of context.
"""

rag_message_list = [
    {"role" : "system", "content" : rag_system_prompt_template},
]

rag_user_prompt_template = """\
Question:
{question}
Context:
{context}
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", rag_system_prompt_template),
    ("human", rag_user_prompt_template)
])

# Generation setup
from langchain_core.globals import set_llm_cache
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o-mini")

# Setting up the cache
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

# RAG LCEL Chain
from operator import itemgetter
from langchain_core.runnables.passthrough import RunnablePassthrough

retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | chat_prompt | chat_model
    )

# Test the chain
retrieval_augmented_qa_chain.invoke({"question" : "Write 50 things about this document!"})

AIMessage(content='1. The document is titled "Use and Care Manual SPE68C75UC | Bosch."\n2. It is authored by BSH Hausgeräte GmbH, located in Munich, Germany.\n3. The document serves as a user manual for the Bosch Dishwasher model SPE68C75UC.\n4. It contains a total of 60 pages.\n5. The document was created on October 10, 2023.\n6. It is formatted as a PDF version 1.4.\n7. The manual includes troubleshooting tips for the dishwasher.\n8. It covers topics related to transportation, storage, and disposal of the appliance.\n9. The document provides guidance on customer service contacts.\n10. It details technical specifications of the dishwasher.\n11. There is a section on the warranty for the appliance.\n12. The document addresses the AquaStop® Plus Pledge for leak prevention.\n13. It includes information about the model number (E-Nr.) and production number (FD).\n14. Safety instructions are prominently featured.\n15. The manual emphasizes the importance of reading all instructions before u

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/batch HTTP/11" 202 33
DEBUG:urllib3.con

We'll define our chunking strategy.

We'll chunk our uploaded PDF file.

#### QDrant Vector Database - Cache Backed Embeddings

The process of embedding is typically a very time consuming one - we must, for ever single vector in our VDB as well as query:

1. Send the text to an API endpoint (self-hosted, OpenAI, etc)
2. Wait for processing
3. Receive response

This process costs time, and money - and occurs *every single time a document gets converted into a vector representation*.

Instead, what if we:

1. Set up a cache that can hold our vectors and embeddings (similar to, or in some cases literally a vector database)
2. Send the text to an API endpoint (self-hosted, OpenAI, etc)
3. Check the cache to see if we've already converted this text before.
  - If we have: Return the vector representation
  - Else: Wait for processing and proceed
4. Store the text that was converted alongside its vector representation in a cache of some kind.
5. Return the vector representation

Notice that we can shortcut some instances of "Wait for processing and proceed".

Let's see how this is implemented in the code.

In [12]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import CacheBackedEmbeddings

# Typical Embedding Model
core_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Typical QDrant Client Set-up
collection_name = f"pdf_to_parse_{uuid.uuid4()}"
client = QdrantClient(":memory:")
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

# Adding cache!
store = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings, store, namespace=core_embeddings.model
)

# Typical QDrant Vector Store Set-up
vectorstore = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=cached_embedder)
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})

##### ❓ Question #1:

What are some limitations you can see with this approach? When is this most/least useful. Discuss with your group!

Based on the context provided in the document, here are some suggested answers for Questions 1 and 2:

Question #1: Limitations and usefulness of cache-backed embeddings

Limitations:
1. Storage requirements: Caching embeddings requires additional storage space, which could become significant for large datasets.
2. Staleness: If the underlying embedding model is updated, cached embeddings may become outdated.
3. Initial overhead: The first-time embedding process still incurs the full cost and time.
4. Cache management: Implementing and maintaining the cache adds complexity to the system.

Most useful:
1. For frequently accessed documents or queries, reducing API calls and processing time.
2. In applications with limited bandwidth or high latency to embedding services.
3. When working with static datasets that don't change frequently.

##### 🏗️ Activity #1:

Create a simple experiment that tests the cache-backed embeddings.

In [13]:
import time
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

# Set up the embeddings and cache
core_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
store = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings, store, namespace=core_embeddings.model
)

# Test text
test_text = "This is a sample text to test cache-backed embeddings."

# Function to measure embedding time
def time_embedding(embedder, text):
    start_time = time.time()
    _ = embedder.embed_query(text)
    end_time = time.time()
    return end_time - start_time

# First embedding (should take longer as it's not cached)
print("First embedding (not cached):")
first_time = time_embedding(cached_embedder, test_text)
print(f"Time taken: {first_time:.4f} seconds")

# Second embedding (should be faster due to cache)
print("\nSecond embedding (should be cached):")
second_time = time_embedding(cached_embedder, test_text)
print(f"Time taken: {second_time:.4f} seconds")

# Calculate and print the speedup
speedup = (first_time - second_time) / first_time * 100
print(f"\nSpeedup: {speedup:.2f}%")

First embedding (not cached):
Time taken: 0.2249 seconds

Second embedding (should be cached):
Time taken: 0.3181 seconds

Speedup: -41.47%


### Augmentation

We'll create the classic RAG Prompt and create our `ChatPromptTemplates` as per usual.

In [14]:
from langchain_core.prompts import ChatPromptTemplate

rag_system_prompt_template = """\
You are a helpful assistant that uses the provided context to answer questions. Never reference this prompt, or the existance of context.
"""

rag_message_list = [
    {"role" : "system", "content" : rag_system_prompt_template},
]

rag_user_prompt_template = """\
Question:
{question}
Context:
{context}
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", rag_system_prompt_template),
    ("human", rag_user_prompt_template)
])

### Generation

Like usual, we'll set-up a `ChatOpenAI` model - and we'll use the fan favourite `gpt-4o-mini` for today.

However, we'll also implement...a PROMPT CACHE!

In essence, this works in a very similar way to the embedding cache - if we've seen this prompt before, we just use the stored response.

In [15]:
from langchain_core.globals import set_llm_cache
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o-mini")

Setting up the cache can be done as follows:

In [16]:
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

##### ❓ Question #2:

What are some limitations you can see with this approach? When is this most/least useful. Discuss with your group!

Limitations:
1. Lack of context sensitivity: Cached responses may not account for subtle changes in context or user intent.
2. Potential for outdated information: If the LLM is updated or fine-tuned, cached responses may become obsolete.
3. Storage requirements: Storing a large number of prompt-response pairs can be memory-intensive.
4. Reduced adaptability: Relying heavily on cached responses may limit the system's ability to provide novel or adaptive answers.

Most useful:
1. For frequently asked questions or common queries with stable answers.
2. In applications requiring quick response times, such as customer service chatbots.
3. To reduce costs associated with repeated API calls to language models.
4. For providing consistent answers to standard queries across multiple users.

##### 🏗️ Activity #2:

Create a simple experiment that tests the cache-backed embeddings.

In [17]:
import time
from langchain_openai import ChatOpenAI
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache

# Set up the LLM and cache
chat_model = ChatOpenAI(model="gpt-3.5-turbo")
set_llm_cache(InMemoryCache())

# Test prompt
test_prompt = "What is the capital of France?"

# Function to measure response time
def time_llm_response(model, prompt):
    start_time = time.time()
    _ = model.invoke(prompt)
    end_time = time.time()
    return end_time - start_time

# First response (should take longer as it's not cached)
print("First response (not cached):")
first_time = time_llm_response(chat_model, test_prompt)
print(f"Time taken: {first_time:.4f} seconds")

# Second response (should be faster due to cache)
print("\nSecond response (should be cached):")
second_time = time_llm_response(chat_model, test_prompt)
print(f"Time taken: {second_time:.4f} seconds")

# Calculate and print the speedup
speedup = (first_time - second_time) / first_time * 100
print(f"\nSpeedup: {speedup:.2f}%")

First response (not cached):
Time taken: 0.4625 seconds

Second response (should be cached):
Time taken: 0.0010 seconds

Speedup: 99.79%


## Task 3: RAG LCEL Chain

We'll also set-up our typical RAG chain using LCEL.

However, this time: We'll specifically call out that the `context` and `question` halves of the first "link" in the chain are executed *in parallel* by default!

Thanks, LCEL!

In [18]:
from operator import itemgetter
from langchain_core.runnables.passthrough import RunnablePassthrough

retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | chat_prompt | chat_model
    )

Let's test it out!

In [19]:
retrieval_augmented_qa_chain.invoke({"question" : "Write 50 things about this document!"})

AIMessage(content='1. The document is a Use and Care Manual for a Bosch dishwasher model SPE68C75UC.\n2. The document contains troubleshooting information on page 16.\n3. Transportation, storage, and disposal instructions are on page 17.\n4. Customer service details are provided on page 18.\n5. Technical specifications are listed on page 19.\n6. The document includes a statement of limited product warranty on page 20.\n7. Information about the warranty coverage and duration is outlined.\n8. There is a section on extended warranty options.\n9. Repair and replacement options are discussed as exclusive remedies.\n10. The document mentions out-of-warranty products.\n11. Colored coatings inside the dishwasher are mentioned on page 45.\n12. The difficulty of removing coatings from stainless steel dishware is highlighted.\n13. The formation of films in the dishwasher is attributed to substances in vegetables and tap water.\n14. Instructions for cleaning the appliance to remove coatings are pr

##### 🏗️ Activity #3:

Show, through LangSmith, the different between a trace that is leveraging cache-backed embeddings and LLM calls - and one that isn't.

Post screenshots in the notebook!

In [20]:

# Set up the cache
cache = InMemoryCache()

# Add cache to the chain
cached_chain = retrieval_augmented_qa_chain.with_config(configurable={"cache": cache})

# Function to measure response time
def time_chain_response(chain, input_data):
    start_time = time.time()
    result = chain.invoke(input_data)
    end_time = time.time()
    return result, end_time - start_time

# Test input
test_input = {"question": "Write 50 things about this document!"}

# First run (not cached)
print("First run (not cached):")
first_result, first_time = time_chain_response(cached_chain, test_input)
print(f"Time taken: {first_time:.4f} seconds")

# Second run (should be cached)
print("\nSecond run (should be cached):")
second_result, second_time = time_chain_response(cached_chain, test_input)
print(f"Time taken: {second_time:.4f} seconds")

# Calculate and print the speedup
speedup = (first_time - second_time) / first_time * 100
print(f"\nSpeedup: {speedup:.2f}%")

# Compare results
print("\nResults comparison:")
print("First run result:")
print(first_result)
print("\nSecond run result:")

First run (not cached):
Time taken: 0.2019 seconds

Second run (should be cached):
Time taken: 0.1976 seconds

Speedup: 2.16%

Results comparison:
First run result:
content='1. The document is a Use and Care Manual for a Bosch dishwasher model SPE68C75UC.\n2. The document contains troubleshooting information on page 16.\n3. Transportation, storage, and disposal instructions are on page 17.\n4. Customer service details are provided on page 18.\n5. Technical specifications are listed on page 19.\n6. The document includes a statement of limited product warranty on page 20.\n7. Information about the warranty coverage and duration is outlined.\n8. There is a section on extended warranty options.\n9. Repair and replacement options are discussed as exclusive remedies.\n10. The document mentions out-of-warranty products.\n11. Colored coatings inside the dishwasher are mentioned on page 45.\n12. The difficulty of removing coatings from stainless steel dishware is highlighted.\n13. The formation

In [23]:
import uuid
from datetime import datetime

new_project_name = f"Test_Project_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{uuid.uuid4().hex[:6]}"
try:
    new_project = client.create_project(new_project_name)
    print(f"Successfully created new project: {new_project_name} with ID: {new_project.id}")
except Exception as e:
    print(f"Error creating project: {e}")
    raise

Successfully created new project: Test_Project_20241004_080054_535737 with ID: 66025f6a-3525-4452-9821-01cfc61400c2


In [24]:
try:
    test_run = client.create_run(
        project_name=new_project_name,
        name="Test Run",
        run_type="chain",
        inputs={"question": "Test question"}
    )
    if test_run is None:
        print("create_run returned None")
    else:
        print(f"Successfully created run with ID: {test_run.id}")
except Exception as e:
    print(f"Error creating run: {e}")
    raise

create_run returned None


In [25]:
try:
    runs = list(client.list_runs(project_name=new_project_name, limit=10))
    print(f"Found {len(runs)} runs in the project")
    for run in runs:
        print(f"Run ID: {run.id}, Name: {run.name}")
except Exception as e:
    print(f"Error listing runs: {e}")
    raise

Found 1 runs in the project
Run ID: 693760f5-5093-40cc-b5a5-4c3f929d4b41, Name: Test Run


In [27]:
try:
    run_id = "693760f5-5093-40cc-b5a5-4c3f929d4b41"
    run_details = client.read_run(run_id)
    print(f"Run Details: {run_details}")
except Exception as e:
    print(f"Error fetching run details: {e}")

Run Details: id=UUID('693760f5-5093-40cc-b5a5-4c3f929d4b41') name='Test Run' start_time=datetime.datetime(2024, 10, 4, 12, 1, 7, 826029) run_type='chain' end_time=None extra={'runtime': {'sdk': 'langsmith-py', 'sdk_version': '0.1.121', 'library': 'langsmith', 'platform': 'macOS-14.6.1-arm64-arm-64bit', 'runtime': 'python', 'py_implementation': 'CPython', 'runtime_version': '3.11.9', 'langchain_version': '0.3.0', 'langchain_core_version': '0.3.2'}, 'metadata': {'revision_id': '2ad5c21'}} error=None serialized=None events=[] inputs={'question': 'Test question'} outputs=None reference_example_id=None parent_run_id=None tags=[] session_id=UUID('66025f6a-3525-4452-9821-01cfc61400c2') child_run_ids=None child_runs=None feedback_stats=None app_path='/o/8fd26bf9-f5de-54a5-9d40-07bb2733eb7c/projects/p/66025f6a-3525-4452-9821-01cfc61400c2/r/693760f5-5093-40cc-b5a5-4c3f929d4b41?trace_id=693760f5-5093-40cc-b5a5-4c3f929d4b41&start_time=2024-10-04T12:01:07.826029' manifest_id=None status='pending' p

In [28]:
import time

try:
    new_run = client.create_run(
        project_name="Test_Project_20241004_080054_535737",
        name="New Test Run",
        run_type="chain",
        inputs={"question": "New test question"}
    )
    print(f"New run creation result: {new_run}")
    
    # Wait a moment to allow for any potential lag
    time.sleep(2)
    
    # List runs again
    runs = list(client.list_runs(project_name="Test_Project_20241004_080054_535737", limit=10))
    print(f"Found {len(runs)} runs in the project after new creation")
    for run in runs:
        print(f"Run ID: {run.id}, Name: {run.name}")
except Exception as e:
    print(f"Error in create and list process: {e}")

New run creation result: None
Found 2 runs in the project after new creation
Run ID: c3adacb9-f223-482c-8a03-13f13e57de2b, Name: New Test Run
Run ID: 693760f5-5093-40cc-b5a5-4c3f929d4b41, Name: Test Run


In [ ]:
for example in examples:
    try:
        run = client.create_run(
            project_name=project_name,
            name="QA Evaluation",
            run_type="chain",
            inputs=example.inputs
        )
        if run is None:
            print(f"create_run returned None for input: {example.inputs}")
            # List runs to get the most recent run
            recent_runs = list(client.list_runs(project_name=project_name, limit=1))
            if recent_runs:
                run = recent_runs[0]
                print(f"Using most recent run with ID: {run.id}")
            else:
                print("No recent runs found, skipping this example")
                continue
        
        # Proceed with your evaluation using 'run'
        # ...

    except Exception as e:
        print(f"Error processing example: {e}")

In [57]:
import time
import logging
from datetime import datetime, timezone
from langsmith import Client
from tenacity import retry, stop_after_attempt, wait_fixed
from langchain_openai import ChatOpenAI
from langchain.evaluation import CriteriaEvalChain, StringEvaluator
from langchain.smith import RunEvalConfig

In [60]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

client = Client()

@retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
def create_and_get_run(project_name, inputs):
    try:
        run = client.create_run(
            project_name=project_name,
            name="QA Evaluation",
            run_type="chain",
            inputs=inputs
        )
        if run is None:
            logger.info("Run created, but returned None. Attempting to fetch...")
            time.sleep(1)  # Wait a bit before trying to fetch
            runs = list(client.list_runs(project_name=project_name, limit=1))
            if runs:
                run = runs[0]
                logger.info(f"Retrieved run with ID: {run.id}")
            else:
                raise Exception("Failed to retrieve the created run")
        return run
    except Exception as e:
        logger.error(f"Error in run creation/retrieval: {e}", exc_info=True)
        raise

def setup_evaluators():
    eval_llm = ChatOpenAI(model="gpt-3.5-turbo")
    
    criteria = {
        "relevance": "The response should be highly relevant to the question asked.",
        "completeness": "The response should fully address all aspects of the question.",
        "accuracy": "The information provided in the response should be accurate and factual.",
        "clarity": "The response should be clear, well-structured, and easy to understand."
    }
    
    criteria_evaluator = CriteriaEvalChain.from_llm(
        llm=eval_llm,
        criteria=criteria
    )
    
    eval_config = RunEvalConfig(
        evaluators=[criteria_evaluator],
        custom_evaluators=[],
    )
    
    return eval_config

from datetime import datetime, timezone

def process_example(project_name, example, cached_chain, eval_config):
    run = create_and_get_run(project_name, example.inputs)
    if run is None:
        logger.warning(f"Failed to create or retrieve run for input: {example.inputs}")
        return

    try:
        # Execute the chain
        result = cached_chain.invoke(example.inputs)
        
        # Check if the run still exists and hasn't been completed
        try:
            existing_run = client.read_run(run.id)
            if existing_run.end_time:
                logger.info(f"Run {run.id} has already been completed. Skipping update.")
                return
        except Exception as e:
            logger.warning(f"Error checking run status: {e}")
            # If we can't check the status, we'll attempt to update anyway

        # Update the run with the result
        client.update_run(
            run.id,
            outputs=result,
            end_time=datetime.now(timezone.utc),
        )
        
        # Run evaluators
        for evaluator in eval_config.evaluators:
            if isinstance(evaluator, StringEvaluator):
                eval_result = evaluator.evaluate_strings(
                    prediction=result['text'] if isinstance(result, dict) else str(result),
                    input=example.inputs["question"],
                )
                client.create_feedback(
                    run.id,
                    evaluator.__class__.__name__,
                    score=eval_result.get("score"),
                    comment=eval_result.get("reasoning"),
                )
        
        logger.info(f"Completed evaluation for run {run.id}")
    except Exception as e:
        logger.error(f"Error processing run {run.id}: {e}", exc_info=True)
        try:
            client.update_run(
                run.id,
                error=str(e),
                end_time=datetime.now(timezone.utc),
            )
        except Exception as update_error:
            logger.error(f"Failed to update run {run.id} with error: {update_error}", exc_info=True)

def process_examples_in_batches(project_name, examples, cached_chain, eval_config, batch_size=5):
    for i in range(0, len(examples), batch_size):
        batch = examples[i:i+batch_size]
        for example in batch:
            process_example(project_name, example, cached_chain, eval_config)
        time.sleep(1)  # Add a small delay between batches

def main():
    project_name = "Test_Project_20241004_080054_535737"  # Your project name
    dataset_name = "qa_eval_dataset"  # Your dataset name
    
    examples = list(client.list_examples(dataset_name=dataset_name))
    logger.info(f"Processing {len(examples)} examples")
    
    eval_config = setup_evaluators()
    
    # Assuming cached_chain is your existing chain
    # If not, you'll need to set it up here
    
    process_examples_in_batches(project_name, examples, cached_chain, eval_config)

if __name__ == "__main__":
    main()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /info HTTP/11" 200 485
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /datasets?limit=1&name=qa_eval_dataset HTTP/11" 200 435
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /sessions?limit=1 HTTP/11" 200 733
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /examples?offset=0&inline_s3_urls=True&limit=100&dataset=c992bfb8-4a05-4d62-b44b-76a67620b4b6 HTTP/11" 200 20287
INFO:__main__:Processing 51 examples
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True htt

In [63]:
from langsmith import Client

client = Client()
project_name = "Test_Project_20241004_080054_535737"

# Fetch all runs for the project
runs = client.list_runs(project_name=project_name)

# Aggregate results
results = []
for run in runs:
    feedbacks = client.list_feedback(run_ids=[run.id])
    for feedback in feedbacks:
        results.append({
            "run_id": run.id,
            "score": feedback.score,
            "comment": feedback.comment,
            "criteria": feedback.key  # Assuming the criteria is stored in the key
        })

# Print or process results
for result in results:
    print(f"Run {result['run_id']}: Score: {result['score']}, Criteria: {result['criteria']}")

# Optional: Calculate average score
if results:
    avg_score = sum(result['score'] for result in results if result['score'] is not None) / len(results)
    print(f"Average Score: {avg_score:.2f}")

# Print additional information for debugging
print(f"Number of runs: {len(list(runs))}")
for run in runs:
    print(f"Run ID: {run.id}")
    feedbacks = list(client.list_feedback(run_ids=[run.id]))
    print(f"  Number of feedbacks: {len(feedbacks)}")
    for feedback in feedbacks:
        print(f"    Feedback: Score={feedback.score}, Key={feedback.key}")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): api.smith.langchain.com:443
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /info HTTP/11" 200 485
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /sessions?limit=1&name=Test_Project_20241004_080054_535737&include_stats=False HTTP/11" 200 794
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/query HTTP/11" 200 None
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /feedback?run=43e683f9-5767-4d69-8de6-8949dac127a2&limit=100&offset=0 HTTP/11" 200 2
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /feedback?run=0ae0a575-8541-4f2e-ab84-207cf5153bd8&limit=100&offset=0 HTTP/11" 200 1880
DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "GET /feedback?run=3397a4c6-4607-4abb-9447-4b47f60f050d&limit=100&offset=0 HTT

Run 0ae0a575-8541-4f2e-ab84-207cf5153bd8: Score: 1.0, Criteria: criteriaevalchain
Run 991f92b1-72e5-4eec-a23e-ff6672b8a3af: Score: 1.0, Criteria: criteriaevalchain
Run d6c5c503-5024-4d2f-8c8e-d237ecd2ad56: Score: 1.0, Criteria: criteriaevalchain
Run ff378e64-55c4-4180-aa03-c4f76df0e2d3: Score: 1.0, Criteria: criteriaevalchain
Run ab598092-495f-48ad-b304-43f7de04c7d8: Score: 1.0, Criteria: criteriaevalchain
Run d29a5b3d-76e4-48aa-b312-5d2c9cc21c6c: Score: 1.0, Criteria: criteriaevalchain
Run 5aca01a9-c384-4294-90d8-b72dd29a94b1: Score: 1.0, Criteria: criteriaevalchain
Run 86f41c1b-f7ac-4bf9-b546-055929bdb3b9: Score: 1.0, Criteria: criteriaevalchain
Run ff1728f8-7b85-4c29-b09c-fac865d410c3: Score: 1.0, Criteria: criteriaevalchain
Run 858584a0-97bf-4cfb-8113-df6b847c716a: Score: 1.0, Criteria: criteriaevalchain
Run f0797b70-8a1d-4236-ba35-a078f5b34ddc: Score: 1.0, Criteria: criteriaevalchain
Run 0068815f-8f9c-4d37-88ae-e96f141701da: Score: 1.0, Criteria: criteriaevalchain
Run 8595bda3-4a8

In [ ]:
import time
from langchain_core.caches import InMemoryCache
from langchain_core.runnables import RunnablePassthrough
from langsmith import Client
from langchain.smith import RunEvalConfig
from langchain_openai import ChatOpenAI
from langchain.evaluation import CriteriaEvalChain
import uuid
from datetime import datetime

# Set up the cache
cache = InMemoryCache()

# Add cache to the chain
cached_chain = retrieval_augmented_qa_chain.with_config(configurable={"cache": cache})

# Set up LangSmith client
client = Client()

# Create a small dataset for evaluation
eval_questions = [
    {"question": "Write 50 things about this document!"},
    {"question": "Summarize the main points of the document."},
    {"question": "What are the key topics discussed in this document?"}
]

dataset_name = "qa_eval_dataset"

# Check if the dataset exists
try:
    existing_dataset = client.read_dataset(dataset_name=dataset_name)
    print(f"Using existing dataset: {dataset_name}")
except Exception:
    # If the dataset doesn't exist, create it
    try:
        client.create_dataset(dataset_name, description="QA evaluation dataset")
        print(f"Created new dataset: {dataset_name}")
    except Exception as e:
        print(f"Error creating dataset: {e}")
        raise

# Add examples to the dataset
for item in eval_questions:
    try:
        client.create_example(inputs=item, dataset_name=dataset_name)
    except Exception as e:
        print(f"Error adding example to dataset: {e}")

# Create a ChatOpenAI instance for evaluation
eval_llm = ChatOpenAI(model="gpt-3.5-turbo")

# Define custom criteria
criteria = {
    "relevance": "The response should be highly relevant to the question asked.",
    "completeness": "The response should fully address all aspects of the question.",
    "accuracy": "The information provided in the response should be accurate and factual.",
    "clarity": "The response should be clear, well-structured, and easy to understand."
}

# Create a CriteriaEvalChain
criteria_eval = CriteriaEvalChain.from_llm(
    llm=eval_llm,
    criteria=criteria
)

# Define evaluation config
eval_config = RunEvalConfig(
    evaluators=[criteria_eval],
    custom_evaluators=[],
)

# Create a unique project name
project_name = f"QA_Eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{uuid.uuid4().hex[:6]}"

# Create a new project
try:
    project = client.create_project(project_name)
    print(f"Created new project: {project_name}")
except Exception as e:
    print(f"Error creating project: {e}")
    raise

# Run evaluation on the dataset
try:
    # Get the examples from the dataset
    examples = list(client.list_examples(dataset_name=dataset_name))
    
    for example in examples:
        # Start a run
        try:
            run = client.create_run(
                project_name=project_name,
                name="QA Evaluation",
                run_type="chain",
                inputs=example.inputs
            )
            
            if run is None:
                print(f"Warning: create_run returned None for input: {example.inputs}")
                continue
            
            print(f"Created run with ID: {run.id}")
            
            # Manually execute the chain
            result = cached_chain.invoke(example.inputs)
            
            # Update the run with the result
            client.update_run(
                run.id,
                outputs=result,
                end_time=datetime.utcnow(),
                error=None,
            )
            
            print(f"Updated run {run.id} with result")
            
            # Run evaluators
            for evaluator in eval_config.evaluators:
                eval_result = evaluator.evaluate_strings(
                    prediction=result['text'] if isinstance(result, dict) else str(result),
                    input=example.inputs["question"],
                )
                client.create_feedback(
                    run.id,
                    evaluator.__class__.__name__,
                    score=eval_result.get("score"),
                    comment=eval_result.get("reasoning"),
                )
            
            print(f"Added feedback for run {run.id}")
            
        except Exception as e:
            print(f"Error processing example: {e}")
            if 'run' in locals() and run is not None:
                client.update_run(
                    run.id,
                    error=str(e),
                    end_time=datetime.utcnow(),
                )
    
    print("Evaluation completed.")
    print(f"Project Name: {project_name}")
    print(f"Dataset Name: {dataset_name}")
    print(f"Number of examples processed: {len(examples)}")
    print("\nFor detailed results, check the LangSmith UI.")
    
except Exception as e:
    print(f"Error running evaluation: {e}")
    import traceback
    traceback.print_exc()